In [3]:
#load libraries.  Using a legacy version of Keras because we trained the model on a legacy version

import scipy.misc
import os
import pandas as pd
import pickle
import numpy as np
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

keras = tf.compat.v1.keras
Sequence = keras.utils.Sequence

from PIL import Image
import glob
#from pathlib import Path

2023-06-11 18:52:59.039513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#Input shape of our images and some empty lists that we will populate later

input_shape = (66, 200, 3)
actual_angle=[]
smooth_angle=0
test_ids=[]

In [5]:
# Parameters for datagen.py

params = {'dim': (66,200,3),
          'batch_size': 32,
          'shuffle': True}

In [6]:
#Path for the test images.  These should be in a seperate folder than the training and validation images

path_name="/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages58000plus/"

In [7]:
# This opens the key file and creates two lists, the angle (target) and the ids which is the file name without jpg

f= open("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/SelfDrivingCarNvidiadatakey58000Plus.txt")                                 #read steering angles from disk and preprocess
data = f.read()
data = data.split()
for i in data:                                      #if the node ends with ".jpg" ignore it. It's for collecting angles
    if i[-1]=='g':
 
       i=i[:-4]
       test_ids.append(int(i))
    else:
        actual_angle.append(float(i))    

In [8]:
#These are the actual angles for the training.  We won't use these for training but you can see the actual vs 
#predicted angles as the model trains

labels={}
for i in test_ids:
    labels[i]=float(actual_angle[i-58000])* scipy.pi / 180

In [10]:
# load the model from disk.  This is the model that we created in the other notebook

filename = '/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/models/finalized_model1.sav'
loaded_model = pickle.load(open(filename, 'rb'))



2023-06-11 18:53:59.310723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
#Image of Steering Wheel.  The idea is we have one image of the steering wheel and rotate it based on predicted angle

steering_wheel = cv2.imread("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/SteeringWheelImage/steering_wheel_image.png") #read steering image
#resize it
steering_wheel=cv2.resize(steering_wheel, (270,210))
#get it's shape
rows,cols,level = steering_wheel.shape

In [12]:
#This code loads and read some images that were reserved for testing.  It then uses the 
#model we trained in the other notebook to predict a steering wheel angle.  

X=[]
y=[]


for i in test_ids:
    

    image = cv2.imread("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages58000plus/"+str(i)+".jpg")   #read images from disk
    image=cv2.resize(image[-150:], (200,66))/255 
    
    X = np.asarray(image)
    X = X.reshape(1,66,200,3)

    result = loaded_model.predict(X)*180.0/scipy.pi  

    print("Actual Angle= {} Predicted Angle= {}".format(str(actual_angle[i-58000]),str(result.item())))   # This prints actual vs predicted angles.  I think you will need to convert Actual to Radians if you use it

    #this section just for the smoother rotation of streeing wheel.
    smooth_angle += 0.2 * pow(abs((result - smooth_angle)), 2.0/3.0)*(result - smooth_angle)/abs(result-smooth_angle)
    M=cv2.getRotationMatrix2D((cols/2,rows/2),-smooth_angle.item(),1)

    dst= cv2.warpAffine(steering_wheel, M, (cols,rows))

    #this section saves the image of the steering wheel 
    img=Image.fromarray(dst,'RGB')
    img.save("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/StillImages/SteeringWheel"+str(i)+".jpg")

    

1/1 [==============================] - 0s 167ms/step
Actual Angle= -16.639999 Predicted Angle= -10.341590881347656
1/1 [==============================] - 0s 22ms/step
Actual Angle= -16.639999 Predicted Angle= -18.388748168945312
1/1 [==============================] - 0s 24ms/step
Actual Angle= -16.940001 Predicted Angle= -0.866263210773468
1/1 [==============================] - 0s 23ms/step
Actual Angle= -17.24 Predicted Angle= -1.4365785121917725
1/1 [==============================] - 0s 27ms/step
Actual Angle= -18.450001 Predicted Angle= 2.38112735748291
1/1 [==============================] - 0s 28ms/step
Actual Angle= -19.16 Predicted Angle= -4.7785820960998535
1/1 [==============================] - 0s 28ms/step
Actual Angle= -19.66 Predicted Angle= 1.0525867938995361
1/1 [==============================] - 0s 31ms/step
Actual Angle= -19.969999 Predicted Angle= 3.1124491691589355
1/1 [==============================] - 0s 25ms/step
Actual Angle= -20.17 Predicted Angle= 2.6003243923187

Actual Angle= -17.34 Predicted Angle= 2.3646066188812256
1/1 [==============================] - 0s 27ms/step
Actual Angle= -17.24 Predicted Angle= 2.972476005554199
1/1 [==============================] - 0s 36ms/step
Actual Angle= -16.74 Predicted Angle= 2.975411891937256
1/1 [==============================] - 0s 34ms/step
Actual Angle= -16.440001 Predicted Angle= 2.8178904056549072
1/1 [==============================] - 0s 30ms/step
Actual Angle= -15.73 Predicted Angle= 0.5222588777542114
1/1 [==============================] - 0s 27ms/step
Actual Angle= -15.33 Predicted Angle= 2.4730334281921387
1/1 [==============================] - 0s 28ms/step
Actual Angle= -14.92 Predicted Angle= -0.42910236120224
1/1 [==============================] - 0s 33ms/step
Actual Angle= 0.0 Predicted Angle= 0.3049222528934479
1/1 [==============================] - 0s 30ms/step
Actual Angle= -14.92 Predicted Angle= -5.191737651824951
1/1 [==============================] - 0s 31ms/step
Actual Angle= -14.92 

1/1 [==============================] - 0s 24ms/step
Actual Angle= 1.71 Predicted Angle= 3.194262981414795
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.51 Predicted Angle= 2.674952983856201
1/1 [==============================] - 0s 27ms/step
Actual Angle= 1.01 Predicted Angle= 2.622297763824463
1/1 [==============================] - 0s 26ms/step
Actual Angle= 0.91 Predicted Angle= 3.270705461502075
1/1 [==============================] - 0s 24ms/step
Actual Angle= 0.91 Predicted Angle= 3.0441250801086426
1/1 [==============================] - 0s 30ms/step
Actual Angle= 0.81 Predicted Angle= 2.1958823204040527
1/1 [==============================] - 0s 30ms/step
Actual Angle= 0.91 Predicted Angle= 4.002965927124023
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.11 Predicted Angle= 4.398867607116699
1/1 [==============================] - 0s 27ms/step
Actual Angle= 1.51 Predicted Angle= 4.927962303161621
1/1 [==============================] - 0s 30

1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.82 Predicted Angle= 18.463529586791992
1/1 [==============================] - 0s 23ms/step
Actual Angle= 1.61 Predicted Angle= 24.792875289916992
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.61 Predicted Angle= 10.69225025177002
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.61 Predicted Angle= 9.804463386535645
1/1 [==============================] - 0s 22ms/step
Actual Angle= 1.41 Predicted Angle= -0.40658748149871826
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.21 Predicted Angle= 3.034855842590332
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.11 Predicted Angle= 3.7663073539733887
1/1 [==============================] - 0s 26ms/step
Actual Angle= 1.11 Predicted Angle= 3.341702699661255
1/1 [==============================] - 0s 28ms/step
Actual Angle= 0.91 Predicted Angle= 3.704906702041626
1/1 [==============================] - 0

1/1 [==============================] - 0s 26ms/step
Actual Angle= 0.0 Predicted Angle= -2.398498058319092
1/1 [==============================] - 0s 28ms/step
Actual Angle= -5.34 Predicted Angle= -3.0879030227661133
1/1 [==============================] - 0s 26ms/step
Actual Angle= -5.85 Predicted Angle= -2.416700601577759
1/1 [==============================] - 0s 30ms/step
Actual Angle= -6.66 Predicted Angle= -1.3919366598129272
1/1 [==============================] - 0s 25ms/step
Actual Angle= -7.87 Predicted Angle= -2.144577741622925
1/1 [==============================] - 0s 31ms/step
Actual Angle= -8.87 Predicted Angle= -4.947956562042236
1/1 [==============================] - 0s 24ms/step
Actual Angle= -10.18 Predicted Angle= -1.4278779029846191
1/1 [==============================] - 0s 30ms/step
Actual Angle= -11.39 Predicted Angle= -2.4942173957824707
1/1 [==============================] - 0s 29ms/step
Actual Angle= -11.9 Predicted Angle= -2.9886717796325684
1/1 [==================

...

1/1 [==============================] - 0s 24ms/step
Actual Angle= -152.869995 Predicted Angle= -17.465431213378906
1/1 [==============================] - 0s 25ms/step
Actual Angle= -152.869995 Predicted Angle= -18.60946273803711
1/1 [==============================] - 0s 23ms/step
Actual Angle= -152.869995 Predicted Angle= 22.417367935180664
1/1 [==============================] - 0s 23ms/step
Actual Angle= -152.869995 Predicted Angle= -11.265246391296387
1/1 [==============================] - 0s 22ms/step
Actual Angle= -152.869995 Predicted Angle= 42.24287796020508
1/1 [==============================] - 0s 24ms/step
Actual Angle= 0.0 Predicted Angle= -60.83392333984375
1/1 [==============================] - 0s 24ms/step
Actual Angle= -152.869995 Predicted Angle= -93.98799133300781
1/1 [==============================] - 0s 26ms/step
Actual Angle= -152.869995 Predicted Angle= -67.7834701538086
1/1 [==============================] - 0s 26ms/step
Actual Angle= -152.869995 Predicted Angle= -

1/1 [==============================] - 0s 39ms/step
Actual Angle= 3.13 Predicted Angle= -10.305808067321777
1/1 [==============================] - 0s 37ms/step
Actual Angle= 3.03 Predicted Angle= -10.915244102478027
1/1 [==============================] - 0s 28ms/step
Actual Angle= 2.92 Predicted Angle= -10.13086223602295
1/1 [==============================] - 0s 27ms/step
Actual Angle= 3.03 Predicted Angle= -10.428068161010742
1/1 [==============================] - 0s 41ms/step
Actual Angle= 0.0 Predicted Angle= -8.931607246398926
1/1 [==============================] - 0s 40ms/step
Actual Angle= 2.92 Predicted Angle= -9.475330352783203
1/1 [==============================] - 0s 46ms/step
Actual Angle= 2.92 Predicted Angle= -7.854005336761475
1/1 [==============================] - 0s 40ms/step
Actual Angle= 2.92 Predicted Angle= -10.57189655303955
1/1 [==============================] - 0s 35ms/step
Actual Angle= 2.92 Predicted Angle= -12.180584907531738
1/1 [=============================

1/1 [==============================] - 0s 28ms/step
Actual Angle= 0.4 Predicted Angle= -3.000933885574341
1/1 [==============================] - 0s 36ms/step
Actual Angle= 0.4 Predicted Angle= 1.2188587188720703
1/1 [==============================] - 0s 51ms/step
Actual Angle= 0.4 Predicted Angle= 0.9038589596748352
1/1 [==============================] - 0s 28ms/step
Actual Angle= 0.4 Predicted Angle= -0.009126412682235241
1/1 [==============================] - 0s 30ms/step
Actual Angle= 0.4 Predicted Angle= 0.5268611311912537
1/1 [==============================] - 0s 31ms/step
Actual Angle= 0.5 Predicted Angle= 0.1293172538280487
1/1 [==============================] - 0s 34ms/step
Actual Angle= 0.5 Predicted Angle= 1.068223237991333
1/1 [==============================] - 0s 32ms/step
Actual Angle= 0.5 Predicted Angle= 0.18998084962368011
1/1 [==============================] - 0s 37ms/step
Actual Angle= 0.5 Predicted Angle= 1.7073863744735718
1/1 [==============================] - 0s 3

1/1 [==============================] - 0s 30ms/step
Actual Angle= -2.32 Predicted Angle= -5.313094139099121
1/1 [==============================] - 0s 25ms/step
Actual Angle= -2.32 Predicted Angle= -6.780983924865723
1/1 [==============================] - 0s 30ms/step
Actual Angle= -2.32 Predicted Angle= -5.393526554107666
1/1 [==============================] - 0s 30ms/step
Actual Angle= -2.32 Predicted Angle= -8.037139892578125
1/1 [==============================] - 0s 28ms/step
Actual Angle= -2.32 Predicted Angle= -7.573616027832031
1/1 [==============================] - 0s 30ms/step
Actual Angle= -2.32 Predicted Angle= -8.557448387145996
1/1 [==============================] - 0s 32ms/step
Actual Angle= -2.32 Predicted Angle= -5.215303421020508
1/1 [==============================] - 0s 26ms/step
Actual Angle= -2.22 Predicted Angle= -7.075253009796143
1/1 [==============================] - 0s 27ms/step
Actual Angle= -1.21 Predicted Angle= -6.748729228973389
1/1 [=======================

1/1 [==============================] - 0s 24ms/step
Actual Angle= -8.17 Predicted Angle= -8.436355590820312
1/1 [==============================] - 0s 27ms/step
Actual Angle= -8.17 Predicted Angle= -13.27429485321045
1/1 [==============================] - 0s 23ms/step
Actual Angle= -8.27 Predicted Angle= -12.92319107055664
1/1 [==============================] - 0s 23ms/step
Actual Angle= -8.27 Predicted Angle= -9.213006019592285
1/1 [==============================] - 0s 24ms/step
Actual Angle= -8.47 Predicted Angle= -10.517702102661133
1/1 [==============================] - 0s 24ms/step
Actual Angle= -8.67 Predicted Angle= -10.064252853393555
1/1 [==============================] - 0s 24ms/step
Actual Angle= -8.97 Predicted Angle= -8.862919807434082
1/1 [==============================] - 0s 23ms/step
Actual Angle= -9.38 Predicted Angle= -10.212242126464844
1/1 [==============================] - 0s 25ms/step
Actual Angle= -9.88 Predicted Angle= -10.732824325561523
1/1 [===================

In [13]:
#Makes a Video of the Steering wheel with predictions.  This stitches all the still images of the steering wheel
#rotated based on our predictions into a video and saves it

img_array = []
for i in test_ids:
    img = cv2.imread('/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/StillImages/SteeringWheel'+str(i)+'.jpg')
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)


out = cv2.VideoWriter('/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/steering_wheel_nomanip.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 15, size) 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [15]:
# This takes the still images that we used for predictions and Makes a video of test images road driving

img_array = []
for i in test_ids:
    img = cv2.imread("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages58000plus/"+str(i)+".jpg")
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    
out = cv2.VideoWriter('/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/PredictDrive_nomanip.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
#At this point you have two videos, the steering wheel and the road.  I used Imovie on Mac to overlay the
#steering wheel video on top of the road video.  